# Cooperative Solution of Multi-Regional Transmission Expansion Horizontal Investment Coordination Problem

### This notebook contains codes for the three zone test case (consisting of the IEEE 14 nodes, IEEE 30 nodes, and a 5 node test system for each of the zones) under the assumption that the regional transmission planners all cooperate to achieve the maximum value of the social surplus of the broader region

#### The first code block loads the different package dependencies to create the virtual environment in Julia

In [1]:
import Pkg
Pkg.add("GLPK")
Pkg.add("XLSX")
Pkg.add("SCIP")
Pkg.add("Ipopt")
Pkg.add("Cbc")
using Pkg
using JuMP
using GLPK
using XLSX
using SCIP
using Ipopt
using Cbc
using DataFrames
using DataFrames
import Pkg; Pkg.add("HiGHS")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


#### The next code block reads the different files corresponding to shared candidate lines, internal candidate lines, shared existing lines, internal existing lines, generators, and load

In [2]:
shared_cand = DataFrame(XLSX.readtable("../Input_Data/CandLine.csv", "Taul1", header=true)...) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("../Input_Data/CandLineInt.csv", "Taul1", header=true)...) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("../Input_Data/SharedEline.csv", "Taul1", header=true)...) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("../Input_Data/Tran.csv", "Taul1", header=true)...) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("../Input_Data/Gen.csv" , "Taul1", header=true)...) #Dataframe of generators
load =  DataFrame(XLSX.readtable("../Input_Data/Load.csv", "Taul1", header=true)...) #Dataframe of loads
##An example of the load dataframe can be found as follows

,zoneNum,lNodeID,P_load1,P_load2,P_load3,P_load4
,Any,Any,Any,Any,Any,Any
1,1,2,-21.7,-10.85,-16.275,-5.425
2,1,3,-94.2,-47.1,-70.65,-23.55
3,1,4,-47.8,-23.9,-35.85,-11.95
4,1,5,-7.6,-3.8,-5.7,-1.9
5,1,6,-11.2,-5.6,-8.4,-2.8
6,1,9,-29.5,-14.75,-22.125,-7.375
7,1,10,-9,-4.5,-6.75,-2.25
8,1,11,-3.5,-1.75,-2.625,-0.875
9,1,12,-6.1,-3.05,-4.575,-1.525


#### The following code-section constructs several DataFrames for each zone or region, for loads within a particular zone, generators within a particular zone, shared candidate lines originating from or terminating within a particular zone, internal candidate lines within a particular zone, shared existing lines originating from or terminating within a particular zone, internal existing lines within a particular zone, marginal costs of generators within a particular zone, and two binary vectors for each zone, one for checking which all shared candidate lines belong to a particular zone and the other for checking which all shared existing lines belong to a particular zone.

In [3]:
l(i) = load[load.zoneNum .== i, :] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i

bin_e (generic function with 1 method)

#### Constructing the optimization model with a specified solver

In [4]:
Mod3 = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [5]:
node_number = 3;

In [6]:
@variable(Mod3,0 <= gen_var_1[1:14]) # generation at node 1
@variable(Mod3,0 <= gen_var_2[1:30]) # generation at node 2
@variable(Mod3,0 <= gen_var_3[1:5]) # generation at node 3

@variable(Mod3,shared_line_decision_var_1[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines
@variable(Mod3,shared_line_decision_var_2[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines
@variable(Mod3,shared_line_decision_var_3[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines

@variable(Mod3,int_line_decision_var_1[1:nrow(int_c(1))], Bin) #Decision variable for internal candidate lines of zone 1 (Binary)
@variable(Mod3,int_line_decision_var_2[1:nrow(int_c(2))], Bin) #Decision variable for internal candidate lines of zone 2 (Binary)
@variable(Mod3,int_line_decision_var_3[1:nrow(int_c(3))], Bin) #Decision variable for internal candidate lines of zone 3 (Binary)

@variable(Mod3,shared_cand_flow[1:nrow(shared_cand)])  #Power flowing on shared candidate lines 

@variable(Mod3,int_cand_flow_1[1:nrow(int_c(1))]) #Power flowing on internal candidate lines of zone 1 
@variable(Mod3,int_cand_flow_2[1:nrow(int_c(2))]) #Power flowing on internal candidate lines of zone 2
@variable(Mod3,int_cand_flow_3[1:nrow(int_c(3))]) #Power flowing on internal candidate lines of zone 3

@variable(Mod3,0 <= shared_cand_angle[1:nrow(shared_cand),1:2]) #Phase angle decision for shared candidate lines

@variable(Mod3,0 <= int_cand_angle_1[1:nrow(int_c(1)),1:2]) #Phase angle decision for internal candidate lines of zone 1 
@variable(Mod3,0 <= int_cand_angle_2[1:nrow(int_c(2)),1:2]) #Phase angle decision for internal candidate lines of zone 2
@variable(Mod3,0 <= int_cand_angle_3[1:nrow(int_c(3)),1:2]) #Phase angle decision for internal candidate lines of zone 3 

@variable(Mod3,shared_ex_flow[1:nrow(shared_ex)])  #Power flowing on shared existing shared lines

@variable(Mod3,int_ex_flow_1[1:nrow(int_e(1))])  #Power flowing on internal existing lines of zone 1 
@variable(Mod3,int_ex_flow_2[1:nrow(int_e(2))])  #Power flowing on internal existing lines of zone 2
@variable(Mod3,int_ex_flow_3[1:nrow(int_e(3))])  #Power flowing on internal existing lines of zone 3

@variable(Mod3,0 <= shared_ex_angle[1:nrow(shared_ex),1:2]) #Phase angle for existing shared lines

@variable(Mod3,0 <= int_ex_angle_1[1:nrow(int_e(1)),1:2]) #Phase angle decision for existing internal lines of zone 1
@variable(Mod3,0 <= int_ex_angle_2[1:nrow(int_e(2)),1:2]) #Phase angle decision for existing internal lines of zone 2
@variable(Mod3,0 <= int_ex_angle_3[1:nrow(int_e(3)),1:2]) #Phase angle decision for existing internal lines of zone 3


7×2 Array{VariableRef,2}:
 int_ex_angle_3[1,1]  int_ex_angle_3[1,2]
 int_ex_angle_3[2,1]  int_ex_angle_3[2,2]
 int_ex_angle_3[3,1]  int_ex_angle_3[3,2]
 int_ex_angle_3[4,1]  int_ex_angle_3[4,2]
 int_ex_angle_3[5,1]  int_ex_angle_3[5,2]
 int_ex_angle_3[6,1]  int_ex_angle_3[6,2]
 int_ex_angle_3[7,1]  int_ex_angle_3[7,2]

In [7]:
@expression(Mod3, total_cost , sum((gen_var_1[n] .* sum((g(1).gNodeID .== n) .* MC(1))) for n in 1:14)
    .+ sum((gen_var_2[n] .* sum((g(2).gNodeID .== n) .* MC(2))) for n in 1:30)
    .+ sum((gen_var_3[n] .* sum((g(3).gNodeID .== n) .* MC(3))) for n in 1:5)
    .+ sum(bin_c(1)[c] .* shared_line_decision_var_1[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
    .+ sum(bin_c(2)[c] .* shared_line_decision_var_2[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
    .+ sum(bin_c(3)[c] .* shared_line_decision_var_3[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
    .+ sum(int_line_decision_var_1[c] .* int_c(1).costPerCap[c] .* int_c(1).interestRate[c] 
            .*((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c]) ./ (((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c])-1) for c in 1:nrow(int_c(1)))
    .+ sum(int_line_decision_var_2[c] .* int_c(2).costPerCap[c] .* int_c(2).interestRate[c] 
            .*((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c]) ./ (((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c])-1) for c in 1:nrow(int_c(2)))
    .+ sum(int_line_decision_var_3[c] .* int_c(3).costPerCap[c] .* int_c(3).interestRate[c] 
            .*((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c]) ./ (((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c])-1) for c in 1:nrow(int_c(3))))   

34.30293932 gen_var_1[1] + 55 gen_var_1[2] + 3.6 gen_var_2[1] + 3.15 gen_var_2[2] + 3.5 gen_var_2[13] + 3.667 gen_var_2[22] + 3.75 gen_var_2[23] + 4.375 gen_var_2[27] + 16.30293932 gen_var_3[1] + 37 gen_var_3[2] + 0.2410176361372405 shared_line_decision_var_1[1] + 0.19594024895900847 shared_line_decision_var_1[2] + 0.44297470440637654 shared_line_decision_var_1[3] + 0.6532451422553538 shared_line_decision_var_1[4] + 0.6532451422553538 shared_line_decision_var_2[4] + 0.1533748593600621 shared_line_decision_var_2[5] + 0.34289543567826486 shared_line_decision_var_2[6] + 0.2410176361372405 shared_line_decision_var_3[1] + 0.19594024895900847 shared_line_decision_var_3[2] + 0.44297470440637654 shared_line_decision_var_3[3] + 0.1533748593600621 shared_line_decision_var_3[5] + 0.34289543567826486 shared_line_decision_var_3[6] + 0.1533748593600621 int_line_decision_var_1[1] + 0.34289543567826486 int_line_decision_var_1[2] + 0.2519729832343877 int_line_decision_var_1[3] + 0.21553427385490934 int

In [8]:
#The first zone
for n in 1:14
        # Power balance constraint for each node
    @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n] .+ sum(l(1).P_load1 .* (l(1).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(1) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(1) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(1) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(1) .* shared_ex_flow) .+
        sum((int_c(1).tNodeID1 .== n) .* int_cand_flow_1) .- sum((int_c(1).tNodeID2 .== n) .* int_cand_flow_1) .+
        sum((int_e(1).tNodeID1 .== n) .* int_ex_flow_1) .- sum((int_e(1).tNodeID2 .== n) .* int_ex_flow_1))
        #Lower limit for generation of each node
    @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n] .<= sum((g(1).gNodeID .== n) .* g(1).PgMax))
        #Upper limit for generation of each node
    @constraint(Mod3, sum((g(1).gNodeID .== n) .* g(1).PgMin) .<= sum(g(1).gNodeID .== n) .* gen_var_1[n])
end

In [9]:
#The second zone
for n in 1:30
        # Power balance constraint for each node
    @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n] .+ sum(l(2).P_load1 .* (l(2).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(2) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(2) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(2) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(2) .* shared_ex_flow) .+
        sum((int_c(2).tNodeID1 .== n) .* int_cand_flow_2) .- sum((int_c(2).tNodeID2 .== n) .* int_cand_flow_2) .+
        sum((int_e(2).tNodeID1 .== n) .* int_ex_flow_2) .- sum((int_e(2).tNodeID2 .== n) .* int_ex_flow_2))
        #Lower limit for generation of each node
    @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n] .<= sum((g(2).gNodeID .== n) .* g(2).PgMax))
        #Upper limit for generation of each node
    @constraint(Mod3, sum((g(2).gNodeID .== n) .* g(2).PgMin) .<= sum(g(2).gNodeID .== n) .* gen_var_2[n])
end

In [10]:
#The third zone
for n in 1:5
        # Power balance constraint for each node
    @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n] .+ sum(l(3).P_load1 .* (l(3).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(3) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(3) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(3) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(3) .* shared_ex_flow) .+
        sum((int_c(3).tNodeID1 .== n) .* int_cand_flow_3) .- sum((int_c(3).tNodeID2 .== n) .* int_cand_flow_3) .+
        sum((int_e(3).tNodeID1 .== n) .* int_ex_flow_3) .- sum((int_e(3).tNodeID2 .== n) .* int_ex_flow_3))
        #Lower limit for generation of each node
    @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n] .<= sum((g(3).gNodeID .== n) .* g(3).PgMax))
        #Upper limit for generation of each node
    @constraint(Mod3, sum((g(3).gNodeID .== n) .* g(3).PgMin) .<= sum(g(3).gNodeID .== n) .* gen_var_3[n])
end

In [11]:
#Shared candidate lines
M = 100
for c in 1:nrow(shared_cand)
    @constraint(Mod3,-M .* (1 .- bin_c(1) .* shared_line_decision_var_1[c]) .<= bin_c(1) .* shared_cand_flow[c] .- bin_c(1) .* ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])))
    @constraint(Mod3,-M .* (1 .- bin_c(2) .* shared_line_decision_var_2[c]) .<= bin_c(2) .* shared_cand_flow[c] .- bin_c(2) .* ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])))
    @constraint(Mod3,-M .* (1 .- bin_c(3) .* shared_line_decision_var_3[c]) .<= bin_c(3) .* shared_cand_flow[c] .- bin_c(3) .* ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])))
        
    @constraint(Mod3, bin_c(1) .* shared_cand_flow[c] .- bin_c(1) .* ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])) .<= M .* (1 .- bin_c(1) .* shared_line_decision_var_1[c]))
    @constraint(Mod3, bin_c(2) .* shared_cand_flow[c] .- bin_c(2) .* ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])) .<= M .* (1 .- bin_c(2) .* shared_line_decision_var_2[c]))
    @constraint(Mod3, bin_c(3) .* shared_cand_flow[c] .- bin_c(3) .* ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])) .<= M .* (1 .- bin_c(3) .* shared_line_decision_var_3[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
    @constraint(Mod3, bin_c(1) .* shared_cand_flow[c] .<= bin_c(1) .* shared_line_decision_var_1[c] .*shared_cand.ptMax[c])
    @constraint(Mod3, bin_c(2) .* shared_cand_flow[c] .<= bin_c(2) .* shared_line_decision_var_2[c] .*shared_cand.ptMax[c])
    @constraint(Mod3, bin_c(3) .* shared_cand_flow[c] .<= bin_c(3) .* shared_line_decision_var_3[c] .*shared_cand.ptMax[c])
        #Limiting the lower bound of power flowing within the candidate shared lines
    @constraint(Mod3, -shared_cand.ptMax[c] .* bin_c(1) .* shared_line_decision_var_1[c] .<= bin_c(1) .* shared_cand_flow[c])
    @constraint(Mod3, -shared_cand.ptMax[c] .* bin_c(2) .* shared_line_decision_var_2[c] .<= bin_c(2) .* shared_cand_flow[c])
    @constraint(Mod3, -shared_cand.ptMax[c] .* bin_c(3) .* shared_line_decision_var_3[c] .<= bin_c(3) .* shared_cand_flow[c])
 
end

In [12]:
#Shared existing lines
for h in 1:nrow(shared_ex)
    @constraint(Mod3, shared_ex_flow[h] .== (1 ./ shared_ex.reacT[h]) .* (shared_ex_angle[h,1] .- shared_ex_angle[h,2]))
    @constraint(Mod3, shared_ex_flow[h] .<= shared_ex. ptMax[h])
    @constraint(Mod3, -shared_ex.ptMax[h] .<= shared_ex_flow[h])
    #@constraint(Mod3, (shared_ex.nodeZone1 .== 2) .* (shared_ex.tNodeID1 .== 8) .* shared_ex_angle[h,1] .== 0)
    #@constraint(Mod3, (shared_ex.nodeZone2 .== 2) .*(shared_ex.tNodeID2 .== 8) .* shared_ex_angle[h,2] .== 0)
end

In [13]:
#Zone 1 internal candidate lines
for c in 1:nrow(int_c(1))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_1[c]) .<= int_cand_flow_1[c] .- ((1 ./ int_c(1).reacT[c]) .* (int_cand_angle_1[c,1] .- int_cand_angle_1[c,2])))
    @constraint(Mod3, int_cand_flow_1[c] .- ((1 ./ int_c(1).reacT[c]) .* (int_cand_angle_1[c,1] .- int_cand_angle_1[c,2])) .<= M .* (1 .- int_line_decision_var_1[c]))
    @constraint(Mod3, int_cand_flow_1[c] .<= int_line_decision_var_1[c] .* int_c(1).ptMax[c])
    @constraint(Mod3, -int_c(1).ptMax[c] .*int_line_decision_var_1[c] .<= int_cand_flow_1[c])
end

In [14]:
#Zone 2 internal candidate lines
for c in 1:nrow(int_c(2))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_2[c]) .<= int_cand_flow_2[c] .- ((1 ./ int_c(2).reacT[c]) .* (int_cand_angle_2[c,1] .- int_cand_angle_2[c,2])))
    @constraint(Mod3, int_cand_flow_2[c] .- ((1 ./ int_c(2).reacT[c]) .* (int_cand_angle_2[c,1] .- int_cand_angle_2[c,2])) .<= M .* (1 .- int_line_decision_var_2[c]))
    @constraint(Mod3, int_cand_flow_2[c] .<= int_line_decision_var_2[c] .* int_c(2).ptMax[c])
    @constraint(Mod3, -int_c(2).ptMax[c] .*int_line_decision_var_2[c] .<= int_cand_flow_2[c])
    #@constraint(Mod3, (int_c(2).tNodeID1 .== 8) .* int_cand_angle_2[c,1] .== 0)
    #@constraint(Mod3, (int_c(2).tNodeID2 .== 8) .* int_cand_angle_2[c,2] .== 0)
end

In [15]:
#Zone 3 internal candidate lines
for c in 1:nrow(int_c(3))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_3[c]) .<= int_cand_flow_3[c] .- ((1 ./ int_c(3).reacT[c]) .* (int_cand_angle_3[c,1] .- int_cand_angle_3[c,2])))
    @constraint(Mod3, int_cand_flow_3[c] .- ((1 ./ int_c(3).reacT[c]) .* (int_cand_angle_3[c,1] .- int_cand_angle_3[c,2])) .<= M .* (1 .- int_line_decision_var_3[c]))
    @constraint(Mod3, int_cand_flow_3[c] .<= int_line_decision_var_3[c] .* int_c(3).ptMax[c])
    @constraint(Mod3, -int_c(3).ptMax[c] .*int_line_decision_var_3[c] .<= int_cand_flow_3[c])
end

In [16]:
#Zone 1 internal existing lines
for h in 1:nrow(int_e(1))
    @constraint(Mod3, int_ex_flow_1[h] .== (1 ./ int_e(1).reacT[h]) .* (int_ex_angle_1[h,1] .- int_ex_angle_1[h,2]))
    @constraint(Mod3, int_ex_flow_1[h] .<= int_e(1).ptMax[h])
    @constraint(Mod3, -int_e(1).ptMax[h] .<= int_ex_flow_1[h])
end

In [17]:
#Zone 2 internal existing lines
for h in 1:nrow(int_e(2))
    @constraint(Mod3, int_ex_flow_2[h] .== (1 ./ int_e(2).reacT[h]) .* (int_ex_angle_2[h,1] .- int_ex_angle_2[h,2]))
    @constraint(Mod3, int_ex_flow_2[h] .<= int_e(2).ptMax[h])
    @constraint(Mod3, -int_e(2).ptMax[h] .<= int_ex_flow_2[h])
    #@constraint(Mod3, (int_e(2).tNodeID1 .== 8) .* int_ex_angle_2[h,1] .== 0)
    #@constraint(Mod3, (int_e(2).tNodeID2 .== 8) .* int_ex_angle_2[h,2] .== 0)
end

In [18]:
#Zone 3 internal existing lines
for h in 1:nrow(int_e(3))
    @constraint(Mod3, int_ex_flow_3[h] .== (1 ./ int_e(3).reacT[h]) .* (int_ex_angle_3[h,1] .- int_ex_angle_3[h,2]))
    @constraint(Mod3, int_ex_flow_3[h] .<= int_e(3).ptMax[h])
    @constraint(Mod3, -int_e(3).ptMax[h] .<= int_ex_flow_3[h])
end

In [19]:
@objective(Mod3, Min, total_cost)
optimize!(Mod3)

In [20]:
if termination_status(Mod3) == MOI.OPTIMAL
    shared_line_decision_1 = value.(shared_line_decision_var_1)
    shared_line_decision_2 = value.(shared_line_decision_var_2)
    shared_line_decision_3 = value.(shared_line_decision_var_3)
    shared_cand_power = value.(shared_cand_flow)
    shared_cand_phase_angle = value.(shared_cand_angle)
    shared_ex_power = value.(shared_ex_flow)
    shared_ex_phase_angle = value.(shared_ex_angle)
    int_line_decision_1 = value.(int_line_decision_var_1)
    int_line_decision_2 = value.(int_line_decision_var_2)
    int_line_decision_3 = value.(int_line_decision_var_3)
    obj_value = objective_value(Mod3)
end

2667.2764567175504

In [21]:
shared_line_decision_1

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0

In [22]:
shared_line_decision_2

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0

In [23]:
shared_line_decision_3

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0

In [24]:
shared_cand_power

6-element Array{Float64,1}:
   0.0
   0.0
   0.0
 100.0
   0.0
   0.0

In [25]:
shared_cand_phase_angle

6×2 Array{Float64,2}:
  0.0    0.0
  0.0    0.0
 19.797  0.0
  5.75   0.0
  0.0    0.0
  0.0    0.0

In [26]:
shared_ex_phase_angle

8×2 Array{Float64,2}:
 27.0    0.0
  0.0   32.92
  0.0   20.8
  0.0    0.0
  5.99   0.0
 20.0    0.0
  6.6    0.0
  7.28   0.0

In [27]:
shared_ex_power

8-element Array{Float64,1}:
  100.0
 -100.0
 -100.0
    0.0
  100.0
  100.0
   60.0
   35.0

In [28]:
int_line_decision_1

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [29]:
int_line_decision_2

18-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0

In [30]:
int_line_decision_3

2-element Array{Float64,1}:
 0.0
 0.0

In [31]:
(int_c(1).tNodeID1 .== 7)
(int_c(1).tNodeID2 .== 7)        

10-element BitArray{1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [32]:
bin_c(1)

6-element Array{Int64,1}:
 1
 1
 1
 1
 0
 0

In [33]:
bin_c(2)

6-element Array{Int64,1}:
 0
 0
 0
 1
 1
 1

In [34]:
shared_cand

,Sernum,tNodeID1,nodeZone1,tNodeID2,nodeZone2,reacT,ptMax,lifeTime,interestRate
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,1,1,2,3,0.05917,100,50,0.05
2,2,5,3,2,1,0.22304,100,57,0.045
3,3,10,1,4,3,0.19797,100,29,0.03
4,4,19,2,14,1,0.0575,100,35,0.1
5,5,23,2,4,3,0.1652,100,50,0.05
6,6,2,3,15,2,0.1737,100,57,0.045
